<a href="https://colab.research.google.com/github/TeradaZenichi/dnotools/blob/master/Fluxo_de_carga_radial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fluxo de Carga Radial**

**Nome:** Lucas Zenichi Terada

**Institution:** University of Campinas

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

## Análise Topológica

In [5]:
def verify(df, bar, aux_bus, aux_net, aux_swt, level, radial, line_type):
  current_bar = 0
  if df.start == bar and df.level == -1:
    current_bar = df.end
  if df.end == bar and df.level == -1:
    current_bar = df.start 
  if current_bar > 0:
    if line_type == 'line':
      net.loc[df.num-1, 'level'] = level
      aux_net.append(int(df.num))
    if line_type == 'switch':
      swt.loc[df.num-1, 'level'] = level
      aux_swt.append(int(df.num))
    if bus.loc[current_bar-1, 'level'] == -1:
      bus.loc[current_bar-1, 'level'] = level
      aux_bus.append(int(current_bar))
    else:
      radial = False
  return aux_bus, aux_net, aux_swt, radial

In [6]:
def radiality():
  (level, buslv, netlv, swtlv) = (0, [], [], [])
  aux = []
  (bus['level'], net['level'], swt['level']) = (np.ones(len(bus), dtype=int)*(-1),
  np.ones(len(net), dtype=int)*(-1), np.ones(len(swt), dtype=int)*(-1))
  radial = True
  for ob in bus.itertuples():
    if ob.bustype == 'feeder':
      bus.loc[ob.Index, 'level'] = 0
      aux.append(int(ob.num)) 
  buslv.append(aux)
  netlv.append([])
  swtlv.append([])
  level = 1
  while radial is True:
    aux_bus = []
    aux_net = []
    aux_swt = []
    for bar in buslv[level-1]:
      for ol in net.itertuples():
        (aux_bus, aux_net, aux_swt, radial) =  verify(net.loc[ol.num-1], bar, aux_bus, aux_net, aux_swt, level, radial, 'line')
      for sw in swt.itertuples():
        if sw.pos == 1:
          (aux_bus, aux_net, aux_swt, radial) = verify(swt.loc[sw.Index], bar, aux_bus, aux_net, aux_swt, level, radial, 'switch')
    if len(aux_bus) > 0:
      buslv.append(aux_bus)
      netlv.append(aux_net)
      swtlv.append(aux_swt)
      level +=1
    else:
      break
  return radial, level, buslv, netlv, swtlv

In [7]:
# Carregar os arquivos dataframe para análise topológica
global bus, net, swt, par
bus = pd.read_csv('bus.csv')
net = pd.read_csv('networks.csv')
swt = pd.read_csv('switches.csv')
par = json.load(open('param.json','r'))

In [8]:
net.r = net.r/1000
net.x = net.x/1000
swt.r = swt.r/1000
swt.x = swt.x/1000

## Fluxo de carga - backward forward sweep

In [9]:
def error_calc(v, Ibus):
  error = 0
  for bar in bus.itertuples():
    if bar.bustype == 'load' and bar.level > 0:
      aux = np.abs(bar.pd+1j*bar.qd - v[bar.num-1]*np.conj(Ibus[bar.num-1]))
      error = np.maximum(aux, error)
  return error

In [10]:
def identify(df, lv):
  if bus.level[int(df.start)-1] == int(lv):
    mul = -1
    return int(df.start), mul
  elif bus.level[int(df.end)-1] == int(lv):
    mul = 1
    return int(df.end), mul
  else:
    return 0

In [11]:
def nodalcurrent(Ibus, df, lv):
  if bus.level[int(df.start)-1] == lv:
    return -Ibus[int(df.start)-1]
  elif bus.level[int(df.end)-1] == lv:
    return Ibus[int(df.end)-1]

In [12]:
def current(bar, Ikm, df, nplus):
  Inear = 0
  if bar == df.start:
    Inear = Ikm[nplus-1]
  elif bar == df.end:
    Inear = -Ikm[nplus-1]
  return Inear

In [13]:
def currente_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level):
  for n in netlv[lv]:
    Inet[n-1] = nodalcurrent(Ibus, net.loc[n-1], lv)
  for s in swtlv[lv]:
    Iswt[s-1] = nodalcurrent(Ibus, swt.loc[s-1], lv)
  if lv+1 < level:
    for n in netlv[lv]:
      (bar, mul) = identify(net.loc[n-1], lv)
      for nplus in netlv[lv+1]:
          Inet[n-1] = Inet[n-1] + mul*current(bar, Inet, net.loc[nplus-1], nplus)
      for splus in swtlv[lv+1]:
          Inet[n-1] = Inet[n-1] + mul*current(bar, Iswt, swt.loc[splus-1], splus)
    for s in swtlv[lv]:
      (bar, mul) = identify(swt.loc[s-1], lv)
      for splus in swtlv[lv+1]:
        Iswt[s-1] = Iswt[s-1] + mul*current(bar, Iswt, swt.loc[splus-1], splus)
      for nplus in netlv[lv+1]:
        Iswt[s-1] = Iswt[s-1] + mul*current(bar, Inet, net.loc[nplus-1], nplus)
  return Inet, Iswt

In [14]:
def voltage_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level):
  for n in netlv[lv]:
    if bus.level[net.start[n-1]-1] > bus.level[net.end[n-1]-1]:
      v[net.start[n-1]-1] = v[net.end[n-1]-1] - Inet[n-1]*(net.r[n-1]+1j*net.x[n-1])
    else:
      v[net.end[n-1]-1] = v[net.start[n-1]-1] - Inet[n-1]*(net.r[n-1]+1j*net.x[n-1])
  for s in swtlv[lv]:
      if bus.level[swt.start[s-1]-1] > bus.level[swt.end[s-1]-1]:
        v[swt.start[s-1]-1] = v[swt.end[s-1]-1] - Iswt[s-1]*(swt.r[n-1]+1j*swt.x[n-1])
      else:
        v[swt.end[s-1]-1] = v[swt.start[s-1]-1] - Iswt[s-1]*(swt.r[s-1]+1j*swt.x[s-1])
  return v

In [15]:
def initial_voltage(buslv, level):
  v = np.zeros(len(bus), dtype=complex)
  for bar in buslv[0]:
    v[bar-1] = bus.baseKV[bar-1]/np.sqrt(3)
  for lv in range(1,level):
    for line in netlv[lv]:
      if bus.level[net.start[line-1]-1] < bus.level[net.end[line-1]-1]:
        v[net.end[line-1]-1] = v[net.start[line-1]-1]
      elif bus.level[net.end[line-1]-1] < bus.level[net.start[line-1]-1]:
        v[net.start[line-1]-1] = v[net.end[line-1]-1]
    for switch in swtlv[lv]:
      if bus.level[swt.start[switch-1]-1] < bus.level[swt.end[switch-1]-1]:
        v[swt.end[switch-1]-1] = v[swt.start[switch-1]-1]
      elif bus.level[swt.end[switch-1]-1] < bus.level[swt.start[switch-1]-1]:
        v[swt.start[switch-1]-1] = v[swt.end[switch-1]-1]
  return v

In [16]:
def sweeppowerflow():
  #estados a serem calculados
  Ibus = np.zeros(len(bus), dtype=complex)
  Inet = np.zeros(len(net), dtype=complex)
  Iswt = np.zeros(len(swt), dtype=complex)
  v = initial_voltage(buslv, level)
  error = np.infty
  iteration = 0
  while error > 1e-5 and iteration < 15:
  # Passo 2: Injeção de corrente
    for bar in bus.itertuples():
      if v[bar.num-1] > 0:
        Ibus[bar.num-1] = np.conj((bar.pd+1j*bar.qd)/(par['snom']*v[bar.num-1]))
    # Passo 3: (Backward sweep): Fluxo de corrente para cada level
    for lv in range(level-1,0,-1):
      (Inet, Iswt) = currente_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level)
    # Passo 4: (Forward sweep:) Tensões nas barras
    for lv in range(1,level):
      v = voltage_calc(lv, v, Ibus, Inet, Iswt, buslv, netlv, swtlv, level)
    error = error_calc(v, Ibus)
    iteration = iteration + 1
  return v, Ibus, Inet, Iswt, error, iteration

## Funções adicionais para fluxo de carga

In [17]:
def loss(Inet):
  Ploss = 0
  for line in net.itertuples():
    Ploss = Ploss + line.r*(np.abs(Inet[line.Index]**2))
  return Ploss

In [21]:
def branch_current(Inet, Iswt):
  net['Ikm'] = np.round(np.abs(Inet), decimals=4)
  swt['Ikm'] = np.round(np.abs(Iswt), decimals=4)

In [22]:
def refresh_bus(v):
  v = np.round(np.abs(v), decimals=4)
  bus['v'] = np.abs(v)

In [28]:
def line_power(v, Inet, Iswt):
  S = np.zeros(len(net), dtype=complex)
  for line in net.itertuples():
    S[line.Index] = v[line.start-1]*np.conj(Inet[line.Index])
  net['Pkm'] = np.round(np.real(S), decimals=4)
  net['Qkm'] = np.round(np.imag(S), decimals=4)
  S = np.zeros(len(swt), dtype=complex)
  for line in swt.itertuples():
    S[line.Index] = v[line.start-1]*np.conj(Inet[line.Index])
  swt['Pkm'] = np.round(np.real(S), decimals=4)
  swt['Qkm'] = np.round(np.imag(S), decimals=4)

In [18]:
#swt['pos'] = [1,1,1,0,0,0,0,0,0,0,0,0]
# swt['pos'] = [1,1,0,0,1,0,0,1,1,0,1,1]
swt['pos'] = swt.ini
(radial, level, buslv, netlv, swtlv) = radiality()
if radial is True:
  print("Radial\n")
  (v, Ibus, Inet, Iswt, error, iteration) = sweeppowerflow()

Radial



In [29]:
line_power(v, Inet, Iswt)

,num,start,end,r,x,bsh,tap,Imax,level,Pkm,Qkm
0,1,2,3,0.000092,0.000047,0,1,200,2,1307.8793,504.4069
1,2,3,4,0.000493,0.000251,0,1,200,3,1152.5612,451.0841
2,3,4,5,0.000366,0.000186,0,1,200,4,797.7630,303.8428
3,4,5,6,0.000381,0.000194,0,1,200,5,753.2976,287.0540
4,5,6,7,0.000819,0.000707,0,1,200,6,729.0940,275.3916
5,6,7,8,0.000187,0.000619,0,1,200,7,381.9248,155.8552
6,7,9,10,0.000711,0.000235,0,1,200,9,321.1817,124.7579
7,8,10,11,0.010300,0.000740,0,1,200,10,260.0936,93.5606
8,9,11,13,0.010440,0.000740,0,1,200,11,227.7494,84.1769
9,10,13,14,0.000197,0.000065,0,1,200,12,197.1485,75.6269


array([ 2.97018458e+00+1.14489007e+00j, -1.75719889e+01-5.14400037e+00j,
       -2.54926181e+01-8.32760624e+00j, -3.57418729e+01-1.34655387e+01j,
       -1.04474038e+01-6.45230145e+00j, -1.88872377e+01-4.70991763e+00j,
        1.41348314e+00+5.48418271e-01j, -4.73413629e+01-2.62111189e+01j,
       -9.49364928e+00-1.05996619e+00j, -1.98993520e+01-4.59204453e+00j,
       -1.48778364e+01-9.63992927e+00j, -9.93918849e+00-6.10550684e+00j,
       -1.96316283e+01-1.00007208e+01j, -3.97478201e+01-1.52300532e+01j,
       -1.98510822e+01-5.11875006e+00j, -1.88174258e+01-4.79020209e+00j,
       -1.98496783e+01-5.48488139e+00j, -2.99467570e+01-9.98955219e+00j,
       -2.73368795e+01-9.08712850e+00j, -2.98331401e+01-1.03758998e+01j,
       -2.98490293e+01-1.03929737e+01j, -2.90691971e+01-9.65056385e+00j,
        1.54994998e+00+6.70604346e-01j, -1.39408020e+02-6.03067626e+01j,
       -1.95524290e+01-5.00568475e+00j,  5.03320327e-01+1.77936459e-01j,
       -5.64913000e+00+2.04336472e-02j, -1.88372364